# Experiment: msgspec for dlt as replacement for orjson

One of the requirements for DLT's JSON serializer/deserializer is the use of a `custom_encode` function. Although the `msgspec` package offers an option to extend [1] its existing encoding functionality, it does not appear to support custom encoding for natively supported types [2].

The following code is an investigation into DLT's test: `test_json.py::test_json_named_tuple`


In [179]:
from dlt.common import Decimal
from dlt.common.typing import AnyFun

from typing import Any, NamedTuple 

import msgspec.json

class NamedTupleTest(NamedTuple):
    str_field: str
    dec_field: Decimal

def custom_encode(obj: Any) -> Any:
    if isinstance(obj, tuple):
        return obj._asdict()
    elif hasattr(obj, "asdict"):
        return obj.asdict()
    elif hasattr(obj, "_asdict"):
        return obj._asdict()
    raise TypeError(repr(obj) + " is not JSON serializable")
        

In [180]:
def dumps(
    obj: Any, default: AnyFun = custom_encode
) -> bytes:
    return msgspec.json.encode(obj, enc_hook=default).decode("utf-8")

### Test: expected encoding as dict fails

In [181]:
dumps(NamedTupleTest("STR", Decimal("1.3333"))) == '{"str_field":"STR","dec_field":"1.3333"}'

False

### Conclusion: msgspec supported native types cannnot be custom encoded

In [182]:
dumps(NamedTupleTest("STR", Decimal("1.3333")))

'["STR","1.3333"]'

In [183]:
NamedTupleTest("STR", Decimal("1.3333"))._asdict()

{'str_field': 'STR', 'dec_field': Decimal('1.3333')}

In [184]:
isinstance(NamedTupleTest("STR", Decimal("1.3333")), tuple)

True

### References
- [1] https://jcristharif.com/msgspec/extending.html
- [2] https://jcristharif.com/msgspec/supported-types.html